In [1]:
import json
import os

import numpy as np
import torch

import trainUtils


/home/tyfei/anaconda3/envs/esm3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open(os.path.join("/home/tyfei/evoModel/checkpoints/backbone_lr/", "config.json"), "r") as f:
    configs = json.load(f)

In [3]:
pretrain_model = trainUtils.loadPretrainModel(configs)
model = trainUtils.buildModel(configs, pretrain_model, None) 

In [4]:
a = torch.load("/home/tyfei/evoModel/checkpoints/backbone_lr/last.ckpt")
model.load_state_dict(a["state_dict"], strict=False)

_IncompatibleKeys(missing_keys=['esm_model.encoder.sequence_embed.weight', 'esm_model.encoder.plddt_projection.weight', 'esm_model.encoder.plddt_projection.bias', 'esm_model.encoder.structure_per_res_plddt_projection.weight', 'esm_model.encoder.structure_per_res_plddt_projection.bias', 'esm_model.encoder.structure_tokens_embed.weight', 'esm_model.encoder.ss8_embed.weight', 'esm_model.encoder.sasa_embed.weight', 'esm_model.encoder.function_embed.0.weight', 'esm_model.encoder.function_embed.1.weight', 'esm_model.encoder.function_embed.2.weight', 'esm_model.encoder.function_embed.3.weight', 'esm_model.encoder.function_embed.4.weight', 'esm_model.encoder.function_embed.5.weight', 'esm_model.encoder.function_embed.6.weight', 'esm_model.encoder.function_embed.7.weight', 'esm_model.encoder.residue_embed.weight', 'esm_model.transformer.blocks.0.attn.layernorm_qkv.0.weight', 'esm_model.transformer.blocks.0.attn.layernorm_qkv.0.bias', 'esm_model.transformer.blocks.0.attn.layernorm_qkv.1.linear.w

In [5]:
model.only_embed = True
model.tf = -1.0

In [6]:
import pickle 
with open("/data/tyfei/datasets/covid/S_aln.pkl", "rb") as f:
    S = pickle.load(f)
with open("/data/tyfei/datasets/covid/NSP5_aln.pkl", "rb") as f:
    NSP5 = pickle.load(f)
with open("/data/tyfei/datasets/covid/E_aln.pkl", "rb") as f:
    E = pickle.load(f)
    

In [22]:
S[0]["seq_t"] = torch.tensor(S[0]["seq_t"])

In [7]:
from tqdm import tqdm
cnt = 0
res = []
with torch.no_grad():
    model = model.cuda(0)
    for s, e, nsp5 in tqdm(zip(S, E, NSP5)):
        ret = {}
        s["seq_t"] = torch.tensor(s["seq_t"]).cuda(0)
        ret["S"] = model(s).squeeze().cpu().numpy() 
        e["seq_t"] = torch.tensor(e["seq_t"]).cuda(0)
        ret["E"] = model(e).squeeze().cpu().numpy() 
        nsp5["seq_t"] = torch.tensor(nsp5["seq_t"]).cuda(0)
        ret["NSP5"] = model(nsp5).squeeze().cpu().numpy() 
        res.append(ret) 
        cnt += 1 
        if (cnt % 1000) == 0:
            break
        

999it [05:49,  2.86it/s]


In [ ]:
with open("/data/tyfei/datasets/covid/teststage2_1000.pkl", "wb") as f:
    pickle.dump(res, f)

In [23]:
model(S[0]).shape

torch.Size([1, 256])